사용자 정의 함수


- - -
- 함수기능 : 모델 학습 진행 함수  
- 함수이름 : training  
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료  
            * 모델인스턴스  
            * 학습데이터셋 : feature, target (학습전 Tensor화)  
            * 손실함수인스턴스   
            * 최적화인스턴스  
            * 학습횟수 : EPOCH  
            * BATCH_SIZE  
            * 배치 개수 : BATCH_CNT  
            * 검증용 데이터셋 : feature, target (Tensor화)
- 함수결과 : EPOCH당  Loss율과 성능지표값, 검증의 손실값과 성능지표 값  
- 필요한 요소 : model, feature, target, optimizer, loss_func, EPOCH, BATCH
- - -

---
- 함수기능 : EPOCH 단위 모델 학습 진행 함수  
- 함수이름 : epochtraining  
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료  
            *모델 인스턴스  
            *학습용 데이터셋 : featrue & target(학습 전 Tensor)  
            *손실 함수 인스턴스   
            *최적화 인스턴스  
            *배치 크기 : BATCH_SIZE
            *배치 횟수 : BATCH_CNT  
- 함수결과 : loss값, 성능지표 값
---

In [2]:
### 모듈로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [3]:
# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## 에포크 당 학습 진행 후 손실 값과 성능지표값 반환 함수

def epochTraining(model, feature, target, lossFunc, optimizer, BATCH_CNT, BATCH_SIZE=32, is_reg=True):
    # 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행

    loss_total, score_total = 0, 0


    for batch in range(BATCH_CNT):
        # 배치크기만 큼 데이터셋 추출 및 텐서와 진행
        start = batch *BATCH_SIZE
        end = start + BATCH_SIZE

        x_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)

        #학습 진행
        pre_y=model(x_train).to(DEVICE)

        #손실 계산
        loss=lossFunc(x_train, y_train)
        loss_total +=loss

        #성능 값 추출
        score = R2Score()(pre_y, y_train) if is_reg else R2Score()(pre_y, y_train)
        score_total += score

        # 최적화 : weight, bias 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        # x_val = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        # y_val = torch.FloatTensor(feature[start:end].values).to(DEVICE)

        # if is_reg:
        #     F1Score(pre_y, y_train)
        
        # 에폭당 Loss & 성능지표 값 반환
        # 테스트 및 검증 함수 사용
    return loss_total/BATCH_CNT, score_total/BATCH_CNT





In [ ]:
# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## 검증 및 테스트 진행 후 손실 값과 성능지표값 반환 함수

def testing(model, feature, target, lossFunc, scoreFunc):
    # 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행

    loss_total, score_total = 0, 0


    with torch.no_grad():
        # 텐서화 진행

        featureTS = torch.FloatTensor(feature.values).to(DEVICE)
        targetTS = torch.FloatTensor(target.values).to(DEVICE)

        #학습 진행
        pre_y=model(featureTS).to(DEVICE)

        #손실 계산
        loss=lossFunc(pre_y, targetTS)
       

        #성능 값 추출
        score = scoreFunc(pre_y, targetTS)

    return loss, score

In [ ]:
# 학습 진행
EPOCHS = 10
TV_loss = {'TRAIN': [], 'VAL': []}
TV_score = {'TRAIN': [], 'VAL': []}

for epoch in range(EPOCHS):
    # 학습진행
    train_loss, train_score = epochTraining()

    # 검증진행행
    val_loss, val_score = testing()

    # 에폭당 학습 및 검증 결과 저장
    TV_loss['TRAIN'].append(train_loss)
    TV_score['TRAIN'].append(train_score)
    TV_loss['TRAIN'].append(val_loss)
    TV_score['TRAIN'].append(val_score)

    # 에폭당 학습 및 검증 결과 출력
    print(f'[{epoch}.{EPOCHS}]\n-[TRAIN] LOSS : {train_loss}    SCORE : {train_score}')
    print(f'-[VALID] LOSS : {val_loss}    SCORE : {val_score}')
    